<a href="https://colab.research.google.com/github/smpark0520/ESAA/blob/main/%EA%B0%90%EA%B7%A4_%EC%B0%A9%EA%B3%BC%EB%9F%89_%EC%98%88%EC%B8%A1_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset Info.

- train.csv [파일]
  - ID : 과수나무 고유 ID
  - 착과량(int) : 실제 감귤 착과량 (Target)
  - 나무 생육 상태 Features (5개) : 수고(m), 수관폭1(min), 수관폭2(max), 수관폭평균(수관폭1과 수관폭2의 평균)

      데이터 기입은 cm 단위

  - 새순 Features (89개) : 2022년 09월 01일 ~ 2022년 11월 28일에 일별 측정된 새순 데이터

  - 엽록소 Features (89개)
2022년 09월 01일 ~ 2022년 11월 28일에 일별 측정된 엽록소 데이터



 - test.csv [파일]
  - ID : 과수나무 고유 ID
  - 나무 생육 상태 Features (5개) : 수고(m), 수관폭1(min), 수관폭2(max), 수관폭평균(수관폭1과 수관폭2의 평균)

      데이터 기입은 cm 단위

  - 새순 Features (89개) : 2022년 09월 01일 ~ 2022년 11월 28일에 일별 측정된 새순 데이터

  - 엽록소 Features (89개) : 2022년 09월 01일 ~ 2022년 11월 28일에 일별 측정된 엽록소 데이터



 - sample_submission.csv [제출양식]
  - ID : 과수나무 고유 ID
  - 착과량(int) : 예측한 감귤 착과량


In [ ]:
import pandas as pd
import numpy as np
import random
import os

from supervised.automl import AutoML

train_df = pd.read_csv("../data/train.csv")
test_df =  pd.read_csv("../data/test.csv")

target_columns = train_df.filter(regex="새순").columns
train_df["새순mean"]   = train_df[target_columns].mean(axis=1)
train_df["새순std"]    = train_df[target_columns].std(axis=1)
train_df["새순diff"] = train_df["2022-11-28 새순"] - train_df["2022-09-01 새순"]
train_df["새순min"]    = train_df[target_columns].min(axis=1)
train_df["새순max"]    = train_df[target_columns].max(axis=1)

test_df["새순mean"]   = test_df[target_columns].mean(axis=1)
test_df["새순std"]    = test_df[target_columns].std(axis=1)
test_df["새순diff"]   = test_df["2022-11-28 새순"] - test_df["2022-09-01 새순"]
test_df["새순min"]    = test_df[target_columns].min(axis=1)
test_df["새순max"]    = test_df[target_columns].max(axis=1)

X = train_df[train_df.filter(regex="새순").columns]
print(X.columns)
y = train_df.iloc[:, 1]

automl = AutoML(mode="Compete", eval_metric='mae')
automl.fit(X, y)
pred = automl.predict(test_df[test_df.filter(regex="새순").columns])

submission = pd.read_csv("../data/sample_submission.csv")
submission.iloc[:, 1] = pred
submission.to_csv("automl_fixed.csv", index=False)